# reV demo

## 1. Set up the development environment
> **If you wish to do everything yourself, start by following step 1.1 below
> <br>Otherwise, you can skip to step 1.2 for the semi-automated approach**
<hr>

### 1.1 Manual setup
>[!]The following steps must be done in the Command Line Interface (Terminal)
#### 1.1.1 Clone (copy) the source code for rev into the directory "rev-src"
`$ git clone https://github.com/NREL/reV.git rev-src`
#### 1.1.2 Create a "virtual environment"
`$ conda create -n rev-test python=3.8`
#### 1.1.3 Activate the virtual environment
`$ conda activate rev-test`
#### 1.1.4 Change into the directory of the source code
`$ cd rev-src`
#### 1.1.5 Install `reV`
`$ pip install -e .`
#### 1.1.6 Verify that the package was successfully installed
`$ python -m reV`

### 1.2 Automated setup
>This approach uses a script to handle the setup of the development environment.
><br>It performs all the same steps as outlined in step 1.1 above; but you won't have to type as much!
><br>[!]The following steps must be done in the Command Line Interface (Terminal)
#### 1.2.1 Execute the script
`$ ./configure-dev-env.sh`
#### 1.2.2 Activate the virtual environment
`$ conda activate rev-test`

## Working with the Test Data
> `rex` is a package that comes with `reV`. it handles the "behind-the-scenes" for resource data

### Load test data (Rhode Island)
#### Import 'Resource' from the package rex 
> The *class* `Resource` helps us read and process `h5` resource files

In [1]:
from rex import Resource

#### Get the path to the test data directory
*The `.h5` resource file contains the datasets and metadata*
>*Hierarchical Data Format (HDF) [...] [are] designed to store and organize large amounts of data.* [*from [Wikipedia](https://en.wikipedia.org/wiki/Hierarchical_Data_Format)*]
><br>It contains multidimensional arrays of scientific data.

- The test data can be seen at https://github.com/NREL/reV/tree/main/tests/data

In [2]:
# Import the "pathlib" package, which helps us work with file- and directory paths
import pathlib

In [3]:
# Get the path of the project's root directory
project_root = pathlib.Path('.').parent
print(project_root.absolute())

# Get the path of the test data directory (found in the reV source code we cloned in step 1)
test_data_dir = project_root.joinpath('rev-src/tests/data')
print(test_data_dir.absolute())

/home/morten/projects/equinor/plexos/rev-demo
/home/morten/projects/equinor/plexos/rev-demo/rev-src/tests/data


In [4]:
# Get the path of the directory containing the Resource ('h5') files
wtk_dir = test_data_dir.joinpath('wtk')

# Get the path of the directory containing the SAM parameter files
sam_dir = test_data_dir.joinpath('SAM')

In [5]:
# Build a list of all the Resource (h5) files inside the "wtk" directory
resource_files = [filepath for filepath in wtk_dir.iterdir() if filepath.match("*.h5")]
# Let's inspect the first one by printing the filename
res_path = resource_files[0]
print(f"Resource:\t{res_path.name}")

# Builds a list of all the SAM (.csv or .json) files inside the "SAM" directory
sam_files = [filepath for filepath in sam_dir.iterdir() if filepath.match("*.csv") or filepath.match("*.json")]
sam_path = sam_files[0]
print(f"SAM:\t\t{sam_path}")

Resource:	ri_100_wtk_2012.h5
SAM:		rev-src/tests/data/SAM/i_pvwatts_fixed_lat_tilt.json


## Reading and processing a Resource file
We now have a Resource file and a SAM file
We will now use these files as inputs to calculate capacity factors

> `resource_file.datasets` are 2D arrays: time (0 axis) and value (1 axis)
> `resource_file.meta` is equivalent to Project Points examples

In [6]:
# Read and process the Resource file
with Resource(res_path) as resource:
	datasets = resource.datasets
	meta = resource.meta

print(datasets)

['meta', 'pressure_0m', 'pressure_100m', 'pressure_200m', 'relativehumidity_2m', 'temperature_100m', 'temperature_80m', 'time_index', 'winddirection_100m', 'winddirection_80m', 'windspeed_100m', 'windspeed_80m']


## Computing wind capacity factors for a given set of latitude and longitude coordinates

In [7]:
# import the necessary utilities
import pathlib
import numpy as np
from reV.config.project_points import ProjectPoints
from reV.generation.generation import Gen

In [8]:
# Define an array (list) of coordinate pairs (latitude/longitude)
lat_lons = np.array(
    [
        [ 41.25, -71.66],
        [ 41.05, -71.74],
        [ 41.97, -71.78],
        [ 41.65, -71.74],
        [ 41.25, -71.7 ],
        [ 41.05, -71.78]
    ]
)

In [9]:
# Store the paths to the test files we wish to use
project_root = pathlib.Path('.').parent
test_data_dir = project_root.joinpath('rev-src/tests/data')

res_path = str(test_data_dir.joinpath('wtk/ri_100_wtk_2012.h5').absolute())
sam_path = str(test_data_dir.joinpath('SAM/wind_gen_standard_losses_0.json').absolute())

In [10]:
# Set up project points from the coordinates and test files
pp = ProjectPoints.lat_lon_coords(lat_lons, res_path, sam_path)
print(pp)

ProjectPoints for sites 1 through 189


/home/morten/projects/equinor/plexos/rev-demo/rev-src/reV/config/project_points.py:581: UserWarning: WARNING: points are not in sequential order and will be sorted! The original order is being preserved under column "points_order"
  warn(msg)


### We'll run reV-gen for the technology "windpower", using the project points and input files we defined above.

In [11]:
# Finally, we can run 'reV-gen'
generated_results = Gen.reV_run(
    'windpower',    # technology
    pp,             # project points
    sam_path,       # the SAM parameter file
    res_path,       # the resource (h5) file
    max_workers=1,
    out_fpath="./windpower_cf_output_test",     # a file name to save the output as
    output_request=('cf_mean', 'cf_profile')    # the type of output we want
)

It appears old SAM v1 keys are being used. Updated key "wind_farm_losses_percent" to "turb_generic_loss".
/home/morten/projects/equinor/plexos/rev-demo/rev-src/reV/SAM/version_checker.py:101: PySAMVersionWarning: It appears old SAM v1 keys are being used. Updated key "wind_farm_losses_percent" to "turb_generic_loss".
  warn(m, PySAMVersionWarning)


In [12]:
# The generated results are now stored in the variable 'generated_results'
# As seen in the "output_request" above, we requested the output types "cf_mean" and "cf_profile"
# We can display at the generated Capacity Factors "Profile" output by printing the following:
print(generated_results.out["cf_profile"])

[[0.319 0.485 0.6   0.543 0.579 0.486]
 [0.382 0.444 0.288 0.562 0.339 0.601]
 [0.696 0.398 0.5   0.656 0.466 0.677]
 ...
 [0.833 0.833 0.833 0.833 0.833 0.833]
 [0.782 0.833 0.833 0.833 0.833 0.833]
 [0.756 0.833 0.833 0.833 0.833 0.833]]
